In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn.churn_reason_raw AS
SELECT
    h.customerID AS customer_id,
    h.Churn,
    h.CustomerFeedback_clean,
    ai_extract(
        CONCAT(
            'Você é um analista de churn. Leia o feedback e extraia: ',
            '(1) reason_short = motivo principal em 3 a 6 palavras; ',
            '(2) reason_long = explicação em uma frase. ',
            'Feedback: ',
            h.CustomerFeedback_clean
        ),
        array('reason_short', 'reason_long')
    ) AS reason_struct
FROM workspace.churn.history_clean h
WHERE h.Churn = 1;


In [0]:
%sql
SELECT
  customer_id,
  Churn,
  CustomerFeedback_clean,
  reason_struct.reason_short,
  reason_struct.reason_long
FROM workspace.churn.churn_reason_raw
LIMIT 20;

In [0]:
%sql
-- Top motivos curtos
SELECT
  reason_struct,
  COUNT(*) AS qtd
FROM workspace.churn.churn_reason_raw
GROUP BY reason_struct
ORDER BY qtd DESC
LIMIT 50;

Taxonomia com base no Short reason e Long Reason

1. **Better deal / pricing**  
   _Exemplos:_ encontrou oferta melhor, melhor preço, promoção concorrente, custo alto etc.

2. **Technical issues / speed**  
   _Exemplos:_ internet lenta, velocidades inconsistentes, problemas técnicos genéricos.

3. **Service reliability / outages**  
   _Exemplos:_ quedas frequentes, instabilidade, serviço “vai e volta”.

4. **Product / plan mismatch**  
   _Exemplos:_ plano sem internet, recurso esperado não incluído, tipo de serviço não atende uso.

5. **Payment / billing issues**  
   _Exemplos:_ método de pagamento inconveniente, cobrança, faturamento.

6. **Personal reasons**  
   _Exemplos:_ mudança, motivos pessoais genéricos.

7. **Other / unclear**  
   _Exemplos:_ o que não encaixar claramente acima.

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn.churn_reason_final AS
SELECT
    customer_id,
    Churn,
    CustomerFeedback_clean,
    reason_struct.reason_short AS reason_short,
    reason_struct.reason_long  AS reason_long,
    ai_classify(
        CONCAT(
            'Motivo curto: ', reason_struct.reason_short, '. ',
            'Motivo detalhado: ', reason_struct.reason_long, '. ',
            'Classifique o motivo de churn em UMA das categorias.'
        ),
        array(
            'Better deal / pricing',
            'Technical issues / speed',
            'Service reliability / outages',
            'Product / plan mismatch',
            'Payment / billing issues',
            'Personal reasons',
            'Other / unclear'
        )
    ) AS churn_category
FROM workspace.churn.churn_reason_raw;


In [0]:
%sql
SELECT
  churn_category,
  COUNT(*) AS qtd
FROM workspace.churn.churn_reason_final
GROUP BY churn_category
ORDER BY qtd DESC;

In [0]:
import matplotlib.pyplot as plt

# agrega os dados
df_counts = (
    spark.table("workspace.churn.churn_reason_final")
         .groupBy("churn_category")
         .count()
         .filter("churn_category IS NOT NULL")
         .orderBy("count", ascending=False)
         .toPandas()
)

categories = df_counts["churn_category"]
counts = df_counts["count"]

plt.figure(figsize=(10, 6))
plt.bar(categories, counts)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Número de clientes")
plt.title("Contagem de churn por motivo")
plt.tight_layout()
plt.show()

In [0]:
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

# Tabelas
df_reason = spark.table("workspace.churn.churn_reason_final").alias("r")
df_hist   = spark.table("workspace.churn.history_genie").alias("h")

# Join pela combinação (Churn, CustomerFeedback_clean)
df_base = (
    df_reason
    .join(
        df_hist,
        (F.col("r.CustomerFeedback_clean") == F.col("h.CustomerFeedback_clean")) &
        (F.col("r.Churn") == F.col("h.Churn")),
        "inner"
    )
    .filter(F.col("r.churn_category").isNotNull())
)

In [0]:
df_avg_spend = (
    df_base
    .groupBy("r.churn_category")
    .agg(F.avg("h.TotalCharges").alias("avg_spend"))
    .orderBy(F.col("avg_spend").desc())
    .toPandas()
)

categories = df_avg_spend["churn_category"]
avg_spend  = df_avg_spend["avg_spend"]

plt.figure(figsize=(10, 6))
bars = plt.bar(categories, avg_spend)

plt.ylim(0, max(avg_spend) * 1.15)

# valor médio em cima de cada barra
for bar, value in zip(bars, avg_spend):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{value:,.2f}",
        ha="center",
        va="bottom",
        fontsize=9
    )

plt.xticks(rotation=45, ha="right")
plt.ylabel("Gasto total médio (TotalCharges)")
plt.title("Gasto total médio por motivo de churn")
plt.tight_layout()
plt.show()

In [0]:
df_avg_tenure = (
    df_base
    .groupBy("r.churn_category")
    .agg(F.avg("h.tenure").alias("avg_tenure"))
    .orderBy(F.col("avg_tenure").desc())
    .toPandas()
)

categories = df_avg_tenure["churn_category"]
avg_tenure = df_avg_tenure["avg_tenure"]

plt.figure(figsize=(10, 6))
bars = plt.bar(categories, avg_tenure)

plt.ylim(0, max(avg_tenure) * 1.15)

for bar, value in zip(bars, avg_tenure):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{value:,.1f}",
        ha="center",
        va="bottom",
        fontsize=9
    )

plt.xticks(rotation=45, ha="right")
plt.ylabel("Tenure médio (meses)")
plt.title("Tempo médio de relacionamento por motivo de churn")
plt.tight_layout()
plt.show()

In [0]:
from pyspark.sql import functions as F

df_contract = (
    df_base
    .withColumn(
        "contract_type",
        F.when(F.col("h.Contract_One_year"),  F.lit("One year"))
         .when(F.col("h.Contract_Two_year"),  F.lit("Two year"))
         .otherwise(F.lit("Month-to-month"))
    )
    .groupBy("r.churn_category", "contract_type")
    .count()
    .toPandas()
)

# Pivot para ficar motivo x tipo de contrato
pivot = df_contract.pivot(
    index="churn_category",
    columns="contract_type",
    values="count"
).fillna(0)

pivot = pivot.sort_values(by=pivot.columns.tolist(), ascending=False)

pivot

In [0]:
categories = pivot.index.tolist()
contract_types = pivot.columns.tolist()

x = range(len(categories))
width = 0.25  # largura de cada barra

plt.figure(figsize=(10, 6))

for i, ct in enumerate(contract_types):
    plt.bar(
        [xi + i*width for xi in x],
        pivot[ct].values,
        width,
        label=ct
    )

plt.xticks([xi + width for xi in x], categories, rotation=45, ha="right")
plt.ylabel("Número de clientes churnados")
plt.title("Tipo de contrato por motivo de churn")
plt.legend(title="Tipo de contrato")
plt.tight_layout()
plt.show()

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

# ----- recalcula a tabela, para garantir que está tudo consistente -----
df_reason = spark.table("workspace.churn.churn_reason_final").alias("r")
df_hist   = spark.table("workspace.churn.history_genie").alias("h")

df_base = (
    df_reason
    .join(
        df_hist,
        (F.col("r.CustomerFeedback_clean") == F.col("h.CustomerFeedback_clean")) &
        (F.col("r.Churn") == F.col("h.Churn")),
        "inner"
    )
    .filter(F.col("r.churn_category").isNotNull())
)

df_internet = (
    df_base
    .withColumn(
        "internet_service",
        F.when(F.col("h.InternetService_Fiber_optic"), F.lit("Fiber optic"))
         .when(F.col("h.InternetService_No"),          F.lit("No internet"))
         .otherwise(F.lit("DSL/Other"))
    )
)

df_counts = (
    df_internet
    .groupBy("r.churn_category", "internet_service")
    .count()
    .toPandas()
)

pivot_pct = (
    df_counts
    .pivot(index="churn_category", columns="internet_service", values="count")
    .fillna(0)
)

# converte pra porcentagem por categoria de churn
pivot_pct = pivot_pct.div(pivot_pct.sum(axis=1), axis=0) * 100

# garante ordem fixa das colunas
internet_order = ["DSL/Other", "Fiber optic", "No internet"]
pivot_pct = pivot_pct[internet_order]

# ordena categorias pelo total de churn (opcional)
pivot_pct = pivot_pct.sort_index()

categories = pivot_pct.index.tolist()
x = np.arange(len(categories))
width = 0.25  # largura de cada barra

fig, ax = plt.subplots(figsize=(10, 6))

for i, itype in enumerate(internet_order):
    ax.bar(
        x + (i - 1)*width,                 # desloca cada tipo para o lado
        pivot_pct[itype].values,
        width,
        label=itype
    )

ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=45, ha="right")
ax.set_ylabel("Participação (%)")
ax.set_title("Serviço de internet por categoria de churn (percentual)")
ax.legend(title="Tipo de internet")
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import functions as F

# ============================
# 1) Monta base com motivo + features
# ============================
df_reason = spark.table("workspace.churn.churn_reason_final").alias("r")
df_hist   = spark.table("workspace.churn.history_genie").alias("h")

df_base = (
    df_reason
    .join(
        df_hist,
        (F.col("r.CustomerFeedback_clean") == F.col("h.CustomerFeedback_clean")) &
        (F.col("r.Churn") == F.col("h.Churn")),
        "inner"
    )
    .filter(F.col("r.churn_category").isNotNull())
)

# ============================
# 2) Deriva tipo de pagamento a partir das dummies
# ============================
df_pay = (
    df_base
    .withColumn(
        "payment_type",
        F.when(F.col("h.PaymentMethod_Electronic_check"),      F.lit("Electronic check"))
         .when(F.col("h.PaymentMethod_Mailed_check"),          F.lit("Mailed check"))
         .when(F.col("h.PaymentMethod_Credit_card_automatic"),F.lit("Credit card (automatic)"))
         .otherwise(F.lit("Bank transfer (automatic)"))
    )
)

# ============================
# 3) Tabela de contagem e % por motivo de churn
# ============================
df_counts = (
    df_pay
    .groupBy("r.churn_category", "payment_type")
    .count()
    .toPandas()
)

pivot_pct = (
    df_counts
    .pivot(index="churn_category", columns="payment_type", values="count")
    .fillna(0)
)

# transforma em % dentro de cada motivo de churn
pivot_pct = pivot_pct.div(pivot_pct.sum(axis=1), axis=0) * 100

# ordem fixa dos tipos de pagamento
payment_order = [
    "Electronic check",
    "Mailed check",
    "Credit card (automatic)",
    "Bank transfer (automatic)",
]
pivot_pct = pivot_pct[payment_order]

# ordena motivos pelo total de casos (opcional)
totals = df_counts.groupby("churn_category")["count"].sum().sort_values(ascending=False)
pivot_pct = pivot_pct.loc[totals.index]

print("Percentual de tipo de pagamento por motivo de churn:")
print(pivot_pct.round(1))

# ============================
# 4) Gráfico de barras agrupadas em %
# ============================
categories = pivot_pct.index.tolist()
x = np.arange(len(categories))
width = 0.2  # largura de cada barra

fig, ax = plt.subplots(figsize=(10, 6))

for i, ptype in enumerate(payment_order):
    ax.bar(
        x + (i - (len(payment_order)-1)/2)*width,
        pivot_pct[ptype].values,
        width,
        label=ptype
    )

ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=45, ha="right")
ax.set_ylabel("Participação (%)")
ax.set_title("Tipo de pagamento por motivo de churn")
ax.set_ylim(0, 100)
ax.legend(title="PaymentMethod")

plt.tight_layout()
plt.show()

In [0]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# ----------------------------------------------------
# 1) Função de limpeza igual ao notebook do Marcos
# ----------------------------------------------------
def clean_like_marcos(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # tenure: tratar "unknown", 0 e NaN
    df["tenure"] = df["tenure"].replace("unknown", pd.NA)
    df["tenure"] = pd.to_numeric(df["tenure"], errors="coerce")
    df.loc[df["tenure"] == 0, "tenure"] = pd.NA
    df["tenure"] = df["tenure"].fillna(df["tenure"].median())
    df["tenure"] = df["tenure"].astype(int)

    # PhoneService: yes/no -> 1/0
    df["PhoneService"] = (
        df["PhoneService"]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({"yes": 1, "no": 0})
        .astype(int)
    )

    # MultipleLines: "No phone service" -> "No", depois map para 1/0
    df["MultipleLines"] = (
        df["MultipleLines"]
        .replace({"No phone service": "No"})
        .map({"Yes": 1, "No": 0})
        .astype(int)
    )

    # Colunas de serviço de internet: "No internet service" -> "No", depois 1/0
    internet_cols = [
        "OnlineSecurity", "OnlineBackup", "DeviceProtection",
        "TechSupport", "StreamingTV", "StreamingMovies"
    ]
    for col in internet_cols:
        df[col] = (
            df[col]
            .replace({"No internet service": "No"})
            .map({"Yes": 1, "No": 0})
            .astype(int)
        )

    # Binárias simples
    for col in ["Partner", "Dependents", "PaperlessBilling"]:
        df[col] = df[col].map({"Yes": 1, "No": 0}).astype(int)

    # TotalCharges: numérico + mediana
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
    df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].median())

    # Feedback limpo
    if "CustomerFeedback" in df.columns:
        df["CustomerFeedback"] = df["CustomerFeedback"].fillna("").astype(str)
        df["CustomerFeedback_clean"] = (
            df["CustomerFeedback"]
            .str.lower()
            .str.replace("[^a-zA-Z0-9 ]", "", regex=True)
        )

    # Dummies para categóricas
    cat_cols = ["gender", "InternetService", "Contract", "PaymentMethod"]
    cat_cols = [c for c in cat_cols if c in df.columns]
    if cat_cols:
        df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # Churn string -> 0/1, se existir
    if "Churn" in df.columns:
        df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0}).astype(int)

    return df

In [0]:
# ----------------------------------------------------
# 2) Ler e limpar o history.csv
# ----------------------------------------------------
df_raw = pd.read_csv("./history.csv")
df_clean = clean_like_marcos(df_raw)

# target
y = df_clean["Churn"]

# features (igual ao Marcos: exclui id e textos)
X = df_clean.drop(columns=["customerID", "Churn", "CustomerFeedback", "CustomerFeedback_clean"])

# separa tipos de coluna
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
boolean_cols = X.select_dtypes(include=["bool"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("bool", "passthrough", boolean_cols),
    ]
)

# modelo de churn: LogisticRegression balanceada
clf_churn = LogisticRegression(max_iter=2000, class_weight="balanced")

churn_model = Pipeline([
    ("preprocess", preprocess),
    ("model", clf_churn),
])

# aqui posso treinar direto em todo o dataset
churn_model.fit(X, y)

# mediana das features numéricas (para imputar em novas entradas)
train_median = X.median(numeric_only=True)

print("Modelo de churn treinado.")
print("AUC em holdout (opcional):")

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
churn_model_temp = Pipeline([
    ("preprocess", preprocess),
    ("model", clf_churn),
])
churn_model_temp.fit(X_tr, y_tr)
y_proba_te = churn_model_temp.predict_proba(X_te)[:, 1]
y_pred_te = churn_model_temp.predict(X_te)
print("AUC:", roc_auc_score(y_te, y_proba_te))
print(classification_report(y_te, y_pred_te, digits=3))

In [0]:
from pyspark.sql import functions as F

# ----------------------------------------------------
# 3) Construir base de treino de motivos
# ----------------------------------------------------
reason_pdf = (
    spark.table("workspace.churn.churn_reason_final")
    .select("CustomerFeedback_clean", "Churn", "churn_category")
    .toPandas()
)

# mapeia Churn para 0/1 igual df_clean
reason_pdf["Churn"] = reason_pdf["Churn"].astype(int)

# fica só com churnados com categoria preenchida
reason_pdf = reason_pdf[
    (reason_pdf["Churn"] == 1) &
    reason_pdf["churn_category"].notna()
].copy()

# junta com df_clean (history.csv limpo) pelo texto limpo + churn
df_join = reason_pdf.merge(
    df_clean,
    on=["CustomerFeedback_clean", "Churn"],
    how="inner",
    suffixes=("_reason", "")
)

print("Linhas com motivo de churn e features:", len(df_join))

# features de motivo usam exatamente as mesmas colunas de X (modelo de churn)
X_reason = df_join[X.columns].copy()
y_reason = df_join["churn_category"].copy()

# ----------------------------------------------------
# 4) Modelo de motivo (multi-classe, mesma pipeline)
# ----------------------------------------------------
reason_clf = LogisticRegression(
    max_iter=2000,
    multi_class="auto",
    class_weight="balanced"
)

reason_model = Pipeline([
    ("preprocess", preprocess),
    ("model", reason_clf),
])

reason_model.fit(X_reason, y_reason)

reason_classes = list(reason_model.named_steps["model"].classes_)
print("Modelo de motivos treinado. Classes:")
for c in reason_classes:
    print(" -", c)

In [0]:
# ----------------------------------------------------
# 5) Helpers para preparar uma nova entrada
# ----------------------------------------------------
def prepare_single_example(raw_dict: dict) -> pd.DataFrame:
    """
    raw_dict: dicionário com as MESMAS colunas brutas de history.csv
    (gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, etc.)
    """
    df_raw_single = pd.DataFrame([raw_dict])

    # limpa no mesmo estilo do Marcos
    df_clean_single = clean_like_marcos(df_raw_single)

    # alinha colunas com X (features de treino)
    X_single = df_clean_single.reindex(columns=X.columns, fill_value=0)

    # preenche NaN numéricas com mediana do treino
    for col in train_median.index:
        X_single[col] = X_single[col].fillna(train_median[col])

    # qualquer coisa residual vira 0
    X_single = X_single.fillna(0)

    return X_single

def predict_churn_and_reasons(raw_dict: dict, top_k: int = 3):
    """
    Retorna e imprime:
      - P(churn)
      - P(motivo | churn) para cada categoria
      - P(churn e motivo) = P(churn) * P(motivo | churn)
    """
    X_single = prepare_single_example(raw_dict)

    # probabilidade de churn (classe 1)
    p_churn = float(churn_model.predict_proba(X_single)[:, 1][0])

    # probabilidades condicionais dos motivos
    proba_reason = reason_model.predict_proba(X_single)[0]
    reason_probs = list(zip(reason_classes, proba_reason))
    reason_probs_sorted = sorted(reason_probs, key=lambda t: t[1], reverse=True)

    # probabilidade conjunta churn & motivo
    reason_joint = [(label, p_churn * p_r) for label, p_r in reason_probs_sorted]

    print(f"Probabilidade de churn: {p_churn:.3f}\n")

    print("Top motivos (P(motivo | churn)):")
    for label, p_r in reason_probs_sorted[:top_k]:
        print(f"  - {label}: {p_r:.3f}")

    print("\nTop motivos (P(churn E motivo)):")
    for label, p_joint in reason_joint[:top_k]:
        print(f"  - {label}: {p_joint:.3f}")

    return {
        "p_churn": p_churn,
        "reason_probs": reason_probs_sorted,
        "reason_joint": reason_joint,
    }

In [0]:
exemplo_manual = {
    "customerID": "TESTE-0001",
    "gender": "Male",
    "SeniorCitizen": 0,
    "Partner": "No",
    "Dependents": "No",
    "tenure": "2",                      # string, como no CSV original
    "PhoneService": "Yes",
    "MultipleLines": "No",
    "InternetService": "Fiber optic",
    "OnlineSecurity": "No",
    "OnlineBackup": "No",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "Yes",
    "StreamingMovies": "Yes",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 90.0,
    "TotalCharges": 180.0,
    "CustomerFeedback": "",            # pode deixar vazio, não entra no modelo
    "MonthlyIncome": 5000,             # conforme coluna do history.csv
}

_ = predict_churn_and_reasons(exemplo_manual, top_k=3)